In [ ]:
#Useful tips:

#the year column in 'Movie' table, will have few chracters other than numbers which you need to be preprocessed,
#you need to get a substring of last 4 characters, its better if you convert it as int type,
#ex: CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)

#For almost all the TEXT columns we have show, please try to remove trailing spaces, you need to use TRIM() function

#When you are doing count(coulmn) it won't consider the "NULL" values, you might need to explore other alternatives
#like Count(*)

In [ ]:
import pandas as pd
import sqlite3

from IPython.display import display, HTML

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
conn = sqlite3.connect("/content/drive/MyDrive/Db-IMDB-Assignment.db")

In [ ]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)
tables = tables["Table_Name"].values.tolist()

In [ ]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




In [ ]:
%%time

#Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’
#and year is a leap year) Your query should return director name, the movie name, and the year.

#To determine whether a year is a leap year, follow these steps:
#STEP-1: If the year is evenly divisible by 4, go to step 2. Otherwise, go to step 5.
#STEP-2: If the year is evenly divisible by 100, go to step 3. Otherwise, go to step 4.
#STEP-3: If the year is evenly divisible by 400, go to step 4. Otherwise, go to step 5.
#STEP-4: The year is a leap year (it has 366 days).
#STEP-5: The year is not a leap year (it has 365 days).
#Year 1900 is divisible by 4 and 100 but it is not divisible by 400, so it is not a leap year.


def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results.head(10))
    print(q1_results.shape)
    assert (q1_results.shape == (232,3))

query1 = """ select p.name as director_name, nyg.name as movie_name, nyg.year as year
             from person p join m_director md on p.pid = md.pid
             join 
                (select m.title as name, m.year as year, m.mid
                 from movie m join m_genre mg on m.mid = mg.mid
                 join genre g on mg.gid = g.gid 
                 where g.name like '%Comedy%' and year 
                 in (select year from movie
                     where CAST(SUBSTR(TRIM(year),-4) AS INTEGER)%4 = 0 and CAST(SUBSTR(TRIM(year),-4) AS INTEGER)%100 = 0 and CAST(SUBSTR(TRIM(year),-4) AS INTEGER)%400 = 0
                         or CAST(SUBSTR(TRIM(year),-4) AS INTEGER)%4 = 0 and CAST(SUBSTR(TRIM(year),-4) AS INTEGER)%100 != 0 
                     )
                ) as nyg on md.mid = nyg.mid
               """
grader_1(query1)

       director_name                         movie_name  year
0       Milap Zaveri                         Mastizaade  2016
1       Danny Leiner  Harold & Kumar Go to White Castle  2004
2     Anurag Kashyap                 Gangs of Wasseypur  2012
3       Frank Coraci        Around the World in 80 Days  2004
4      Griffin Dunne             The Accidental Husband  2008
5        Anurag Basu                             Barfi!  2012
6    Gurinder Chadha                  Bride & Prejudice  2004
7         Mike Judge    Beavis and Butt-Head Do America  1996
8   Tarun Mansukhani                            Dostana  2008
9       Shakun Batra                      Kapoor & Sons  2016
(232, 3)
CPU times: user 66.7 ms, sys: 274 µs, total: 66.9 ms
Wall time: 71.7 ms


In [ ]:
%%time
#Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results.head(10))
    print(q2_results.shape)
    assert (q2_results.shape == (17,1))

query2 = """select name from person p
            join (select pid from movie m join m_cast mc on m.mid = mc.mid 
             where m.title = 'Anand' and cast(substr(trim(m.year),-4) as integer) = 1971) tb 
             on trim(p.pid) = trim(tb.pid)
             """
grader_2(query2)

                Name
0   Amitabh Bachchan
1      Rajesh Khanna
2      Sumita Sanyal
3         Ramesh Deo
4          Seema Deo
5     Asit Kumar Sen
6         Dev Kishan
7       Atam Prakash
8      Lalita Kumari
9             Savita
(17, 1)
CPU times: user 258 ms, sys: 3.29 ms, total: 261 ms
Wall time: 269 ms


In [ ]:
%%time
#Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

def grader_3(q3):
    q3_results  = pd.read_sql_query(q3,conn)
    print(q3_results.head(10))
    print(q3_results.shape)
    assert (q3_results.shape == (300,1))

query3 = """select p.name from person p
            where p.pid in
            (
                Select p.pid from Person p 
                join 
                (
                    select trim(mc.PID) PD, mc.MID from M_cast mc 
                    where mc.MID 
                    in 
                        (
                        select mv.MID from Movie mv where CAST(SUBSTR(mv.year,-4) AS Integer)<1970
                        )
                ) r1 
                on r1.PD=p.PID 

                intersect

                Select p.PID from Person p 
                join 
                (
                    select trim(mc.PID) PD, mc.MID from M_cast mc 
                    where mc.MID 
                    in 
                        (
                        select mv.MID from Movie mv where CAST(SUBSTR(mv.year,-4) AS Integer)>1990
                        )
                ) r1 
                on r1.PD=p.PID   
            )
             """
grader_3(query3)

                Name
0       Rishi Kapoor
1   Amitabh Bachchan
2             Asrani
3       Zohra Sehgal
4    Parikshat Sahni
5      Rakesh Sharma
6        Sanjay Dutt
7          Ric Young
8              Yusuf
9     Suhasini Mulay
(300, 1)
CPU times: user 238 ms, sys: 3.34 ms, total: 241 ms
Wall time: 241 ms


In [ ]:
%%time
# Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed.
#Return the directors' names and the number of movies each of them directed.

def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    print(q4_results.shape)
    assert (q4_results.shape == (58,2))

query4 = """ 
             select p.name, count(md.mid) as cnt from person p 
             join
             m_director md 
             on p.pid = md.pid group by p.name having count(md.mid) >= 10 
             order by count(md.mid) desc
             
         """
grader_4(query4)


                    Name  cnt
0           David Dhawan   39
1           Mahesh Bhatt   36
2        Ram Gopal Varma   30
3           Priyadarshan   30
4           Vikram Bhatt   29
5   Hrishikesh Mukherjee   27
6            Yash Chopra   21
7        Basu Chatterjee   19
8         Shakti Samanta   19
9           Subhash Ghai   18
(58, 2)
CPU times: user 30 ms, sys: 523 µs, total: 30.5 ms
Wall time: 30.3 ms


In [ ]:
%%time
# Q5.a --- For each year, count the number of movies in that year that had only female actors.

def grader_5a(q5a):
    q5a_results  = pd.read_sql_query(q5a,conn)
    print(q5a_results.head(10))
    print(q5a_results.shape)
    assert (q5a_results.shape == (4,2))

query5a = """ select trim(m.year) as YEAR, count(m.mid) as Female_Cast_Only_Movies from movie m
             
              where trim(m.mid) in 
                (select trim(mc.mid) from m_cast mc 

                 except
              
                 select distinct trim(mc.mid) from m_cast mc
                 where trim(mc.pid) in
                    (select trim(p.pid) from person p 
                      where p.gender = 'Male')
                
                )
                group by trim(m.year)
          """
grader_5a(query5a)

     YEAR  Female_Cast_Only_Movies
0    1939                        1
1    1999                        1
2    2000                        1
3  I 2018                        1
(4, 2)
CPU times: user 109 ms, sys: 3.43 ms, total: 112 ms
Wall time: 114 ms


In [ ]:
%%time
# Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors,
# and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there
# were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(10))
    assert (q5b_results.shape == (4,3))

query5b = """select tb1.year as YEAR, cast(tb2.Female_Cast_Only_Movies as float) / cast(tb1.Total_movies as float) 
             as Percentage_Female_Only_Movie, tb1.Total_movies 
             from
             (select trim(m.year) as YEAR, count(m.mid) as Total_movies from movie m
               where m.year in
                (select trim(m.year) from movie m
             
                    where trim(m.mid) in 
                      (select trim(mc.mid) from m_cast mc 

                       except
              
                       select distinct trim(mc.mid) from m_cast mc
                       where trim(mc.pid) in
                          (select trim(p.pid) from person p 
                            where p.gender = 'Male')
                
                      )
                      group by trim(m.year)
                ) group by trim(m.year)
              ) as tb1
              
              join
              
              (select trim(m.year) as YEAR, count(m.mid) as Female_Cast_Only_Movies from movie m
             
                  where trim(m.mid) in 
                    (select trim(mc.mid) from m_cast mc 

                     except
              
                     select distinct trim(mc.mid) from m_cast mc
                     where trim(mc.pid) in
                        (select trim(p.pid) from person p 
                          where p.gender = 'Male')
                
                    )
                    group by trim(m.year)
              ) as tb2
              on tb1.year = tb2.year
          """
grader_5b(query5b)

     YEAR  Percentage_Female_Only_Movie  Total_movies
0    1939                      0.500000             2
1    1999                      0.015152            66
2    2000                      0.015625            64
3  I 2018                      0.100000            10
CPU times: user 218 ms, sys: 0 ns, total: 218 ms
Wall time: 218 ms


In [ ]:
%%time
# Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast.
# By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles,
# or if it simply occurs multiple times in casts, we still count her/him only once.

def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    print(q6_results.shape)
    assert (q6_results.shape == (3473, 2))

query6 = """select title, Cast_size from
            (select trim(mc.mid) as MID, count(mc.pid) as Cast_size from m_cast mc 
              where trim(mc.pid) in
               (
                select distinct trim(p.pid) from person p 
            
               )
              group by trim(mc.mid)
              
            ) as tb1
            
            join 
            
            (select trim(m.title) as title, trim(m.mid) as m_mid from movie m 
              where trim(m.mid) in
                (select trim(mc.mid) from m_cast mc 
                 where trim(mc.pid) in
                  (
                   select distinct trim(p.pid) from person p 
            
                  )
                 group by trim(mc.mid)
                )
            ) as tb2
            
            on tb1.MID = tb2.m_mid
            order by Cast_size desc
         """
grader_6(query6)

                        title  Cast_size
0               Ocean's Eight        238
1                    Apaharan        233
2                        Gold        215
3             My Name Is Khan        213
4  Captain America: Civil War        191
5                    Geostorm        170
6                     Striker        165
7                        2012        154
8                      Pixels        144
9       Yamla Pagla Deewana 2        140
(3473, 2)
CPU times: user 293 ms, sys: 7.86 ms, total: 300 ms
Wall time: 307 ms


In [ ]:
%%time
#Q7 --- A decade is a sequence of 10 consecutive years.
#For example, say in your database you have movie information starting from 1931.
#the first decade is 1931, 1932, ..., 1940,
#the second decade is 1932, 1933, ..., 1941 and so on.
#Find the decade D with the largest number of films and the total number of films in D

def grader_7(q7):
    q7_results  = pd.read_sql_query(q7,conn)
    print(q7_results.head(20))
    print(q7_results.shape)
    assert (q7_results.shape == (1, 2))

query7 = """select decade_year as Higest_movie_decade, max(Decade_total) as Movies_count from
             (select cast(substr(trim(year),-4)as integer) as decade_year, sum(Total_movies) as Decade_total from 
               (select * from 
                   (select m.year, count(m.mid) as Total from movie m
                    group by m.year
                    order by m.year
                   ) tb1, 
                   (select m.year, count(m.mid) as Total_movies  from movie m
                    group by m.year
                    order by m.year
                   ) tb2
                   where cast(substr(trim(tb2.year),-4)as integer) <= cast(substr(trim(tb1.year),-4)as integer) + 9
                   and cast(substr(trim(tb1.year),-4)as integer) <= cast(substr(trim(tb2.year),-4)as integer)
               )
              group by year
             )
         """
grader_7(query7)


   Higest_movie_decade  Movies_count
0                 2008          1203
(1, 2)
CPU times: user 16.6 ms, sys: 2 ms, total: 18.6 ms
Wall time: 18.8 ms


In [ ]:
%%time
#Q8a --- Write a query that will results in number of movies actor-director worked together.
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    print(q8a_results.shape)
    assert (q8a_results.shape == (73408, 3))

query8a = """
              select trim(mc.pid) as actor_id, trim(md.pid) as director_id, count(mc.mid) as movie_count 
              from m_cast mc join m_director md 
              on trim(mc.mid) = trim(md.mid)
              group by actor_id, director_id
              order by movie_count desc
            
          
          """
grader_8a(query8a)

    actor_id director_id  movie_count
0  nm0456094   nm0223522           23
1  nm0007106   nm0223522           20
2  nm0434318   nm0223522           20
3  nm0318622   nm0080315           19
4  nm0332871   nm0223522           17
5  nm0712546   nm0698184           16
6  nm2147526   nm0698184           15
7  nm0442479   nm0223522           14
8  nm0451272   nm0080315           14
9  nm0451600   nm0223522           14
(73408, 3)
CPU times: user 51.7 s, sys: 30.4 ms, total: 51.7 s
Wall time: 51.6 s


In [ ]:
%%time 
# Second temporary table contains max movies done by each actor - compute max(count of movies) for each actor

def grader_8b(q8b):
    q8b_results  = pd.read_sql_query(q8b,conn)
    print(q8b_results.head(10))
    print(q8b_results.shape)

query8b = """     
                        select actor_id, max(movie_count) 
                        from 
                            ( 
                              select trim(mc.pid) as actor_id, trim(md.pid) as director_id, count(mc.mid) as movie_count 
                              from m_cast mc join m_director md 
                              on trim(mc.mid) = trim(md.mid)
                              group by actor_id, director_id
                              order by movie_count desc
                            )
                        
                        group by actor_id
                        order by movie_count desc
                     
                    
          
          """
grader_8b(query8b)

    actor_id  max(movie_count)
0  nm0456094                23
1  nm0007106                20
2  nm0434318                20
3  nm0318622                19
4  nm0332871                17
5  nm0712546                16
6  nm2147526                15
7  nm0442479                14
8  nm0451272                14
9  nm0451600                14
(32127, 2)
CPU times: user 51.6 s, sys: 53.1 ms, total: 51.6 s
Wall time: 51.5 s


In [ ]:
%%time
#Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.
def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245, 2))

query8 = """   select p.name as actor_name, tb1.movie_count from person p
               join
                  (        
                     select actor_id, movie_count from
                     (  
                        select trim(mc.pid) as actor_id, trim(md.pid) as director_id, count(mc.mid) as movie_count 
                        from m_cast mc join m_director md 
                        on trim(mc.mid) = trim(md.mid)
                        group by actor_id, director_id
                        order by movie_count desc
                     ) 

                     where (actor_id, movie_count) in

                        (
                            select actor_id, max(movie_count) as movie_count
                            from 
                                ( 
                                  select trim(mc.pid) as actor_id, trim(md.pid) as director_id, count(mc.mid) as movie_count 
                                  from m_cast mc join m_director md 
                                  on trim(mc.mid) = trim(md.mid)
                                  group by actor_id, director_id
                                  order by movie_count desc
                                )
                            group by actor_id
                            order by movie_count desc
                        ) 
                     and director_id like (select trim(p.pid) from person p
                                           where p.name like '%Yash Chopra%' )
                     group by actor_id, movie_count
                 ) tb1
                 on trim(p.pid) = trim(tb1.actor_id)
                 order by tb1.movie_count desc

          """
grader_8(query8)

            actor_name  movie_count
0          Jagdish Raj           11
1     Manmohan Krishna           10
2             Iftekhar            9
3        Shashi Kapoor            7
4       Waheeda Rehman            5
5        Rakhee Gulzar            5
6       Achala Sachdev            4
7          Neetu Singh            4
8             Ravikant            4
9      Parikshat Sahni            3
10       Surendra Rahi            3
11        Sudha Chopra            3
12         Saul George            3
13        Mohan Sherry            3
14       Leela Chitnis            3
15         Yash Chopra            2
16            Raj Hans            2
17       Manohar Singh            2
18      Poonam Dhillon            2
19       Surendra Nath            2
20              Rehman            2
21               Nanda            2
22   Chandu Allahabadi            2
23    Gajanan Jagirdar            2
24         Shyam Arora            2
25           C.L. Shah            2
26         Ashok Verma      

In [ ]:
%%time
#Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in
#the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as 
#Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have 
#Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(30))
    print(q9_results.shape)
    assert (q9_results.shape == (25698, 1))

query9 = """select trim(p.name) from person p
            where trim(p.pid) in
            (
              select trim(mc.pid) from m_cast mc
              where trim(mc.mid) in
              (
                select trim(mc.mid) from m_cast mc
                where trim(mc.pid) in
                (
                   select trim(mc.pid) from m_cast mc
                   where trim(mc.mid) in
                   (
                      select trim(mc.mid) from m_cast mc
                       where trim(mc.pid) in
                           (
                             select trim(p.pid) from person p where p.name like '%Shah rukh khan%'
                           )
                   )
                   
                   
                )
                except
             
                select trim(mc.mid) from m_cast mc
                   where trim(mc.pid) in
                   (
                     select trim(p.pid) from person p where p.name like '%Shah rukh khan%'
                   )
              )
              except
              
              select trim(mc.pid) from m_cast mc
                   where trim(mc.mid) in
                   (
                      select trim(mc.mid) from m_cast mc
                       where trim(mc.pid) in
                           (
                             select trim(p.pid) from person p where p.name like '%Shah rukh khan%'
                           )
                   )
                   
            )
            
         """
grader_9(query9)

              trim(p.name)
0             Freida Pinto
1              Rohan Chand
2             Damian Young
3          Waris Ahluwalia
4    Caroline Christl Long
5            Rajeev Pahuja
6        Michelle Santiago
7          Alicia Vikander
8             Dominic West
9           Walton Goggins
10               Daniel Wu
11    Kristin Scott Thomas
12            Derek Jacobi
13      Alexandre Willaume
14            Tamer Burjaq
15          Adrian Collins
16          Keenan Arrison
17          Andrian Mazive
18           Milton Schorr
19       Hannah John-Kamen
20            Peter Waison
21              Samuel Mak
22                Sky Yang
23             Civic Chung
24             Josef Altin
25     Billy Postlethwaite
26  Roger Jean Nsengiyumva
27          Jaime Winstone
28          Michael Obiora
29           Shekhar Varma
(25698, 1)
CPU times: user 252 ms, sys: 12.9 ms, total: 265 ms
Wall time: 266 ms
